In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt                  # Importing required libraries
%matplotlib inline
import os
import sklearn
import warnings
warnings.filterwarnings("ignore")

#### Importing data

In [4]:
os.chdir("C:/Users/ABC/Desktop/Afgan Forum paper in R/afghanForum")
data=pd.read_csv("afghanForum.csv",error_bad_lines=False)
data.head()

b'Skipping line 38554: expected 12 fields, saw 16\nSkipping line 53649: expected 12 fields, saw 17\nSkipping line 54548: expected 12 fields, saw 20\n'


MessageID  ThreadID                                     ThreadName  \
0    1000015      5536  Love Animals? Like George Clooney?Are hungry?   
1    1000038      5536  Love Animals? Like George Clooney?Are hungry?   
2    1000069      5536  Love Animals? Like George Clooney?Are hungry?   
3    1000099      5536  Love Animals? Like George Clooney?Are hungry?   
4    1000132      5536  Love Animals? Like George Clooney?Are hungry?   

             Date_Time  MemberID                          MemberName  \
0  2009-03-14 17:32:00       339       Paradox\t\t\t\t\t\t\t\t\t\t\t   
1  2009-03-14 18:08:00       240      Filament\t\t\t\t\t\t\t\t\t\t\t   
2  2009-03-14 19:15:00       988  ~*~JiLk@Y~*~\t\t\t\t\t\t\t\t\t\t\t   
3  2009-03-14 22:15:00       339       Paradox\t\t\t\t\t\t\t\t\t\t\t   
4  2009-03-14 22:38:00       240      Filament\t\t\t\t\t\t\t\t\t\t\t   

                                             Message  P_Year  P_Month  P_Day  \
0  I don't like celebrity news but this needed to...    2009        3     14   
1  Ew! "Have courage for the great sorrows of lif...    2009        3     14   
2  what? I am a traveler seeking the truth, a hum...    2009        3     14   
3  Well it is as good a time as any to throw the ...    2009        3     14   
4  Ew X infinity. "Have courage for the great sor...    2009        3     14   

                    P_Date  ThreadFirstMessageID  
0  2009-03-14 17:32:00.000               1000015  
1  2009-03-14 18:08:00.000               1000015  
2  2009-03-14 19:15:00.000               1000015  
3  2009-03-14 22:15:00.000               1000015  
4  2009-03-14 22:38:00.000               1000015

In [5]:
dat=data[["MessageID","Message"]]
dat.shape

(88032, 2)

#### Sentiment Analysis

In [6]:
from textblob import TextBlob
import re
import textblob

In [7]:
def clean_text(text): 
        ''' 
        Utility function to clean text by removing links,
        special characters using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()) 

def get_text_sentiment(text):
    ''' 
    Utility function to classify sentiment of passed
    text using textblob's sentiment method 
    '''
    # create TextBlob object of passed text 
    analysis = TextBlob(clean_text(text)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'    

In [8]:
sentiment = []
for message in dat['Message']:
    sentiment.append(get_text_sentiment(message))
    
dat["Sentiment"] = sentiment
dat.head()

MessageID                                            Message Sentiment
0    1000015  I don't like celebrity news but this needed to...  positive
1    1000038  Ew! "Have courage for the great sorrows of lif...  positive
2    1000069  what? I am a traveler seeking the truth, a hum...  positive
3    1000099  Well it is as good a time as any to throw the ...  positive
4    1000132  Ew X infinity. "Have courage for the great sor...  positive

In [9]:
dat['Sentiment_Score'] = dat.Sentiment.map({'positive':1, 'negative':-1,'neutral':0})
dat.head()

MessageID                                            Message Sentiment  \
0    1000015  I don't like celebrity news but this needed to...  positive   
1    1000038  Ew! "Have courage for the great sorrows of lif...  positive   
2    1000069  what? I am a traveler seeking the truth, a hum...  positive   
3    1000099  Well it is as good a time as any to throw the ...  positive   
4    1000132  Ew X infinity. "Have courage for the great sor...  positive   

   Sentiment_Score  
0                1  
1                1  
2                1  
3                1  
4                1

#### Data Cleaning

#### Converting to lower case

In [10]:
dat['Message'] = [entry.lower() for entry in dat['Message']]
dat.head()

MessageID                                            Message Sentiment  \
0    1000015  i don't like celebrity news but this needed to...  positive   
1    1000038  ew! "have courage for the great sorrows of lif...  positive   
2    1000069  what? i am a traveler seeking the truth, a hum...  positive   
3    1000099  well it is as good a time as any to throw the ...  positive   
4    1000132  ew x infinity. "have courage for the great sor...  positive   

   Sentiment_Score  
0                1  
1                1  
2                1  
3                1  
4                1

#### Removing punctuation and word tokenization

In [11]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
dat['Message']= [tokenizer.tokenize(entry) for entry in dat['Message']]

In [12]:
dat['Message']=[" ".join(content) for content in dat['Message'].values]

#### Removing Stop words

In [13]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
dat['Message']=dat['Message'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
dat.head()

MessageID                                            Message Sentiment  \
0    1000015  like celebrity news needed told news link lett...  positive   
1    1000038  ew courage great sorrows life patience small o...  positive   
2    1000069  traveler seeking truth human searching meaning...  positive   
3    1000099  well good time throw offer paypal account reas...  positive   
4    1000132  ew x infinity courage great sorrows life patie...  positive   

   Sentiment_Score  
0                1  
1                1  
2                1  
3                1  
4                1

#### Model Processing

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [15]:
Y=dat["Sentiment_Score"]

#### Spliting Data in train and test data in 70:30

In [16]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dat['Message'],Y,test_size=0.3)
print(Train_X.shape,Test_X.shape,Train_Y.shape,Test_Y.shape)

(61622,) (26410,) (61622,) (26410,)


#### TFIDF vectorization

In [17]:
Tfidf_vect = TfidfVectorizer(stop_words='english')
Train_X_Tfidf = Tfidf_vect.fit_transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

#### Count Vectorizer

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the training data 
count_train = count_vectorizer.fit_transform(Train_X) 

# Transform the test set 
count_test = count_vectorizer.transform(Test_X)

In [19]:
print(Train_X_Tfidf.shape,Test_X_Tfidf.shape,count_train.shape,count_test.shape)

(61622, 117892) (26410, 117892) (61622, 117892) (26410, 117892)


#### Random Forest  ALgorithm On train  for COunt vectorizer

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
rf.fit(count_train,Train_Y)
predictions_rf = rf.predict(count_test)

print("Random Forest Accuracy Score -> ",accuracy_score(predictions_rf, Test_Y)*100)

Random Forest Accuracy Score ->  67.36463460810299


In [19]:
confusion_matrix(Test_Y, predictions_rf)

array([[    0,     0,  4226],
       [    0,     0,  4393],
       [    0,     0, 17791]], dtype=int64)

#### Random Forest  ALgorithm On train  for TFIDF vectorizer

In [20]:
rf1= RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
rf1.fit(Train_X_Tfidf,Train_Y)
predictions_rf1 = rf1.predict(Test_X_Tfidf)

print("Random Forest Accuracy Score -> ",accuracy_score(predictions_rf1, Test_Y)*100)

Random Forest Accuracy Score ->  67.36463460810299


In [21]:
confusion_matrix(Test_Y, predictions_rf1)

array([[    0,     0,  4226],
       [    0,     0,  4393],
       [    0,     0, 17791]], dtype=int64)

#### Logistic Regression ALgorithm On train  for Count vectorizer

In [22]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression(random_state=0)
log.fit(count_train,Train_Y)
predictions_log= log.predict(count_test)

print("Logistic Regression Accuracy Score -> ",accuracy_score(predictions_log, Test_Y)*100)

Logistic Regression Accuracy Score ->  90.20825444907233


In [23]:
print(classification_report(Test_Y, predictions_log))

              precision    recall  f1-score   support

          -1       0.82      0.71      0.76      4226
           0       0.83      0.93      0.88      4393
           1       0.94      0.94      0.94     17791

   micro avg       0.90      0.90      0.90     26410
   macro avg       0.86      0.86      0.86     26410
weighted avg       0.90      0.90      0.90     26410



In [24]:
confusion_matrix(Test_Y, predictions_log)

array([[ 2999,   352,   875],
       [   67,  4095,   231],
       [  574,   487, 16730]], dtype=int64)

#### Logistic Regression ALgorithm On train  for TFIDF vectorizer

In [25]:
log1=LogisticRegression(random_state=0)
log1.fit(Train_X_Tfidf,Train_Y)
predictions_log1= log1.predict(Test_X_Tfidf)

print("Logistic Regression Accuracy Score -> ",accuracy_score(predictions_log1, Test_Y)*100)

Logistic Regression Accuracy Score ->  86.74744414994319


In [26]:
print(classification_report(Test_Y, predictions_log1))

              precision    recall  f1-score   support

          -1       0.89      0.51      0.65      4226
           0       0.81      0.84      0.82      4393
           1       0.88      0.96      0.92     17791

   micro avg       0.87      0.87      0.87     26410
   macro avg       0.86      0.77      0.80     26410
weighted avg       0.87      0.87      0.86     26410



In [27]:
confusion_matrix(Test_Y, predictions_log1)

array([[ 2151,   410,  1665],
       [   42,  3671,   680],
       [  227,   476, 17088]], dtype=int64)

#### KNN ALgorithm On train  for Count vectorizer

In [28]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(count_train,Train_Y)
knn.score(count_train,Train_Y)

#### KNN ALgorithm On train  for TFIDF vectorizer

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(Train_X_Tfidf,Train_Y)
knn.score(Train_X_Tfidf,Train_Y)

#### GBM ALgorithm On train  for Count vectorizer

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

In [21]:
gbm = GradientBoostingClassifier(random_state=10)
gbm.fit(count_train,Train_Y)
gbm.score(count_train,Train_Y)

0.8048424264061537

In [22]:
prediction_gbm = gbm.predict(count_test)

print("Confusion Matrix:")
print(confusion_matrix(Test_Y, prediction_gbm))


Confusion Matrix:
[[ 1174   660  2578]
 [    9  3216  1131]
 [  259   770 16613]]


In [23]:
print("Classification Report")
print(classification_report(Test_Y, prediction_gbm))

Classification Report
              precision    recall  f1-score   support

          -1       0.81      0.27      0.40      4412
           0       0.69      0.74      0.71      4356
           1       0.82      0.94      0.88     17642

   micro avg       0.80      0.80      0.80     26410
   macro avg       0.77      0.65      0.66     26410
weighted avg       0.80      0.80      0.77     26410



#### GBM ALgorithm On train  for TFIDF vectorizer

In [24]:
gbm1 = GradientBoostingClassifier(random_state=10)
gbm1.fit(Train_X_Tfidf,Train_Y)
gbm1.score(Train_X_Tfidf,Train_Y)

0.8098081853883353

In [25]:
prediction_gbm1 = gbm1.predict(Test_X_Tfidf)

print("Confusion Matrix:")
print(confusion_matrix(Test_Y, prediction_gbm1))

Confusion Matrix:
[[ 1627  1191  1594]
 [   14  3808   534]
 [  621  1865 15156]]


In [26]:
print("Classification Report")
print(classification_report(Test_Y, prediction_gbm1))

Classification Report
              precision    recall  f1-score   support

          -1       0.72      0.37      0.49      4412
           0       0.55      0.87      0.68      4356
           1       0.88      0.86      0.87     17642

   micro avg       0.78      0.78      0.78     26410
   macro avg       0.72      0.70      0.68     26410
weighted avg       0.80      0.78      0.77     26410

